# Convert embeddings to sqlite

In [1]:
from gensim.models import KeyedVectors
from gensim import models
from IPython.display import Markdown
import pandas as pd
import os
import sqlite3

In [157]:
m = KeyedVectors.load('appfolder/models/digibok_1800-1900.kv')

In [160]:
for w in list(m.key_to_index.keys())[:1]:
    print(w, m[w], len(m[w]))
    

, [-2.0834186e+00  2.3842187e+00 -1.1359137e+00  9.5092350e-01
  4.8095706e-01 -3.4627026e-01 -1.9660989e+00 -2.6519382e+00
 -5.8226073e-01 -2.9285338e+00 -2.8516545e+00 -1.0839516e+00
 -2.3576317e+00 -3.6351082e+00 -2.5283656e+00  2.6835694e+00
  1.7565607e+00  1.5337167e+00 -5.5487948e-03  5.3154498e-01
  2.0355337e+00 -1.4429165e+00 -2.7451189e+00  3.5180764e+00
 -5.5195236e-01  2.6215389e+00  2.6553817e+00  3.5855339e+00
 -6.7578959e-01  5.3025928e+00  2.2660626e-01  6.8641968e+00
 -2.6604261e+00 -2.7738145e-01  1.4060694e-01 -3.2409801e+00
  1.6026145e+00 -1.4253813e+00 -4.2587938e+00 -1.8931340e+00
 -3.0287647e+00 -1.2368945e+00 -7.7408171e-01 -3.3569664e-01
  1.6961468e+00 -1.7526623e+00 -3.3612483e+00 -1.5947877e+00
  7.3459077e-01 -4.0351087e-03 -8.3057398e-01 -6.5627420e-01
  8.2659179e-01  5.8815271e-01 -1.7935346e-01  1.2279378e+00
  6.4008367e-01 -3.3737750e+00 -1.9322957e+00 -2.1493955e+00
 -4.1261654e+00 -5.2205038e-01 -2.4300472e-01 -1.0062777e+00
  3.8657901e-01  3.159

In [158]:
db = "vss_1800.db"

In [84]:
with sqlite3.connect(db) as con:
    con.execute("drop table words")

In [159]:
with sqlite3.connect(db) as con:
    con.execute("create table words (word string, embedding blob)")

In [55]:
import sqlite_vss

In [161]:
with sqlite3.connect(db) as con:
    con.enable_load_extension(True)
    sqlite_vss.load(con)
    con.execute("""create virtual table word_vectors using vss0(
          vector(100),
);""")

In [162]:
%%time
with sqlite3.connect(db) as con:
    for w in list(m.key_to_index.keys()): 
        cur = con.cursor()
        cur.execute("""insert into words values (?, ?)""", (w, m[w].tobytes()))

CPU times: user 10.2 s, sys: 1.7 s, total: 11.9 s
Wall time: 12 s


In [77]:
with sqlite3.connect('test_vss.db') as con:
    cur = con.cursor()
    a = cur.execute("""select * from words order by random() limit 10""").fetchall()

In [44]:
import numpy

In [ ]:
%%time
with sqlite3.connect(db) as con:
    con.enable_load_extension(True)
    sqlite_vss.load(con)
    con.execute("""insert into word_vectors (rowid, vector) select rowid, embedding from words""")

In [142]:
def nearest(word, limit=20):
    with sqlite3.connect('test_vss1.db') as con:
        con.enable_load_extension(True)
        sqlite_vss.load(con)
        try:
            res = con.execute("""
            select b.word, a.distance from 
                (
                select rowid, distance from word_vectors
                where 
                    vss_search(
                        vector, 
                        (select vector from word_vectors where rowid = (select rowid from words where word = ?)))
                    limit ?
                ) as a, 
                words as b 
                where 
                    a.rowid = b.rowid""", (word, limit)).fetchall()
        except:
            res = []
    return res

In [150]:
gcon = sqlite3.connect('test_vss1.db')
gcon.enable_load_extension(True)
sqlite_vss.load(gcon)

In [151]:
def similar(con, word, limit = 20):
    try:
        res = con.execute("""
        select b.word, a.distance from 
            (
            select rowid, distance from word_vectors
            where 
                vss_search(
                    vector, 
                    (select vector from word_vectors where rowid = (select rowid from words where word = ?)))
                limit ?
            ) as a, 
            words as b 
            where 
                a.rowid = b.rowid""", (word, limit)).fetchall()
    except:
        res = []
    return res

In [156]:
%%time
similar(gcon, "pølsene")

CPU times: user 37.3 ms, sys: 9.78 ms, total: 47 ms
Wall time: 30.4 ms


[('pølsene', 0.0),
 ('bollene', 8.992934226989746),
 ('nøttene', 10.1032075881958),
 ('rekene', 10.536861419677734),
 ('brødskivene', 10.69122314453125),
 ('pannekakene', 10.780485153198242),
 ('pærene', 11.154183387756348),
 ('ertene', 13.058422088623047),
 ('baconet', 13.067215919494629),
 ('skinken', 13.101675033569336),
 ('jordbærene', 13.543998718261719),
 ('linsene', 13.64631462097168),
 ('flesket', 13.87731647491455),
 ('steken', 13.976338386535645),
 ('plommene', 14.128902435302734),
 ('syltetøyet', 14.311455726623535),
 ('fiskestykkene', 14.739439010620117),
 ('blåskjellene', 14.739874839782715),
 ('gulrøttene', 14.96258544921875),
 ('lefsene', 15.438885688781738)]

In [149]:
%%time
nearest('ost',30)

CPU times: user 304 ms, sys: 249 ms, total: 554 ms
Wall time: 533 ms


[('ost', 0.0),
 ('skinke', 40.014244079589844),
 ('pasta', 45.5068473815918),
 ('kylling', 46.75252151489258),
 ('hvitvin', 48.86185836791992),
 ('potet', 49.592281341552734),
 ('bacon', 50.66096496582031),
 ('biff', 50.88382339477539),
 ('flatbrød', 52.060977935791016),
 ('ris', 52.51927947998047),
 ('sjokolade', 52.922088623046875),
 ('asparges', 53.113929748535156),
 ('sirup', 53.4311637878418),
 ('majones', 53.66023254394531),
 ('syltetøy', 53.889442443847656),
 ('kjøttdeig', 54.02251434326172),
 ('rødvin', 54.688636779785156),
 ('flesk', 54.697288513183594),
 ('suppe', 56.01039505004883),
 ('tomat', 56.31900405883789),
 ('saus', 57.1614990234375),
 ('yoghurt', 57.59467315673828),
 ('kål', 57.68960952758789),
 ('sennep', 58.03306579589844),
 ('kaviar', 58.97991180419922),
 ('parmesan', 59.10783767700195),
 ('pølse', 59.138397216796875),
 ('bringebær', 59.27168655395508),
 ('jordbær', 59.43475341796875),
 ('agurk', 60.73179626464844)]

In [120]:
with sqlite3.connect('test_vss1.db') as con:
    con.enable_load_extension(True)
    sqlite_vss.load(con)
    res = con.execute("""select * from word_vectors limit 2""").fetchall()
    

In [121]:
res

[(b'\r\xe9\x9c?&\xa3\x13\xc0o\xa6\t\xc0(\xea\x10@Rp\x8c\xbf\x12\x86P?\xf9\x97e?^\xe74\xbf\xbd\xb1\xfc\xbe)k\x84\xbf\xff\xf2\xfc>\x86t\x19@^\xde\x9f\xbf\x06\x06\xfb\xbf\xa2\x05\n\xc0{\x97\xbb\xbed\xbc\x90\xbe\r\xfa\xc0?\xfa\xe9\xed=q\xef\xad\xbf\x0c\x08\x06=\xfa\x17\x80=>3s?\xf0\tE\xbf\xca5\xda?.r\xb6\xbf\xbf)a?\xb5\x13u@z\x80\xa1\xbfS\xc4\x10?\xa9E\x93?\xf6x\xce>\xbf\xd6a\xbf(\\\x19\xc0\xd7\x88\x8c\xbf\xec\xb89\xbf\xdd\xa3\xc9\xbf4u\x91\xbf\xd0\xbc\x0c\xc0\x0e\xcd\x8f?\x99\xd4G?\x02\x8b\x18\xc0{6g\xbf\xa3X\x03\xc0U\xc9\x01>\xfd\x80\xef\xbe1C\xab=\xc2\x82\x08@\xeb\x9b\x18@e\xc8C>',),
 (b'Tv\xbb\xbf\xed\xb0/\xc0u\xe9\xb3\xbf"\xce\x00?\xc0i\xfa\xbf\xfb\x95\x7f\xc0k\xf6\xf2\xbf\xe4\x0b\x0c?/\x81\x00\xbf\xe1\x98\x0c?\x0f\xf9:\xbfG\x93\x94?o\xc38\xc0I|~\xc0d\x13i\xbff\x9b\xb3\xbf\x97\xcb\xb4\xbf\xd5F\x8f?s\x03\x1b?\'ee\xc0\xebn\x0e\xbfz\x90\x8c\xbf\x85\xdb\xc7\xbeq}I?=\xf8\xe8?\x08\xd4j\xc0C\x1b?@\x07O|@\x0fg\x89=\x98\xdd\xc0\xbf\x99\r\xe5?\x1ew\x80?\xfbf\t\xc0E\x88\x11?\xc3\x17c\xbf\xef\x1c

In [92]:
with sqlite3.connect('test_vss0.db') as con:
    res = con.execute("select * from sqlite_master").fetchall()

In [93]:
res

[('table',
  'words',
  'words',
  2,
  'CREATE TABLE words (word string, embedding blob)'),
 ('table',
  'word_vectors',
  'word_vectors',
  0,
  'CREATE VIRTUAL TABLE word_vectors using vss0(\n        word string,\n          vector(200),\n)')]

In [80]:
len(a[0][1])

200